<h1><center>STAT/CSE 416 Section 5: Precision and Recall, Random Forests, Ensemble Methods</center></h1>
<center><b>Section:</b>AA/AB BA/BB</center>
<center><b>Instructor:</b>Emilija Perković</center>
<center><b>TA:</b>Octavian-Vlad Murad</center>
<center><b>Date:</b>February 9, 2023</center>

*Author: Anne Wagner. Modifications: Octavian-Vlad Murad*


# University of Wisconsin Breast Cancer Data Set

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as colors

We have talked a bit about precision and recall, different types of classification error, and when certain types of error might be preferred. To help motivate this with an example, we will consider data on breast cancer scans by the University of Wisconsin.

Data Source: UCI Machine Learning Repository http://archive.ics.uci.edu/ml

The data we will be looking at consists of different metrics gathered from breast cancer biopsies. In total there are 10 different metrics gathered about the biopsied cells, repeated on 3 perpendicular axes.

In [ ]:
from sklearn.datasets import load_breast_cancer # load dataset 
dataset = load_breast_cancer() # returns a dictionary

print("Target names:", dataset.target_names)
print("Feature names:", dataset.feature_names)

X_columns = list(dataset.feature_names)

pd_dataset = pd.DataFrame(dataset.data, columns=list(dataset.feature_names))
pd_dataset["cancer type"] = -((dataset.target - 0.5) * 2).astype(int) # transform 0, 1 labels to -1, 1 labels. Flip labels because malignant is 0.

pd_dataset.head()

For the purposes of this exercise, we are going to focus on just two of the variables: <i>perimeter error</i> and <i>worst texture</i>, chosen to help visualize examples. Note that we are not going to be using pd.Dataframe objects, but numpy arrays. This is no reason to worry because sklearn works with both just as fine. Furthermore, a pd.Dataframe object is pretty much a numpy array(the data in the table) + a bunch of features useful for data analysis that can manipulate that data.

In [ ]:
# transform the data to numpy
y = pd_dataset["cancer type"].to_numpy()
X = pd_dataset[["perimeter error", "worst texture"]].to_numpy()

# generate some random indices and split the data into train/test data at the mid point
n = y.shape[0]
inds = np.arange(n)
np.random.RandomState(416)
np.random.shuffle(inds)
train_idx, test_idx = inds[:(n//2)], inds[(n//2):]

X_train, y_train = X[train_idx], y[train_idx]
X_test, y_test = X[test_idx], y[test_idx]

# boolean array of which samples are positives.
mal = (y == 1)
mal_test = (y_test == 1)
mal_train = (y_train == 1)

Again for the purposes visualization, we are going to convert the data to a log scale. This will have no effect on the classification via decision trees (and related methods). 

In [ ]:
X = np.log(X)
X_train = np.log(X_train)
X_test = np.log(X_test)

### Question 1:
- Why would using the log of the features have no effect on the classification when using decision trees?
- Which other types of transformations could we apply to the data(i.e. what property should the transormations have to preserve the classification of a decision tree)?

### Answer:
- Decision trees only branch on a single variable at a time and the logarithm function is one-to-one. A branch at $x<a$ is identical to a branch at $log(x)<log(a)$ .
- The transformations should be one-to-one or equivalently strictly monotonous(increasing or decreasing).

Let us now visualize the data:

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 7.5), dpi=80)

axs[0].set_title("Raw Data")
axs[0].set_xlabel("perimeter error")
axs[0].set_ylabel("worst texture")
axs[0].scatter(np.exp(X[mal, 0]),np.exp(X[mal, 1]),marker='+',cmap="blue")
axs[0].scatter(np.exp(X[~mal, 0]),np.exp(X[~mal, 1]),marker='_',cmap="red")

axs[1].set_title("Log Transformed Data")
axs[1].set_xlabel("log perimeter error")
axs[1].set_ylabel("log worst texture")
axs[1].scatter(X[mal, 0], X[mal, 1],marker='+',cmap="blue")
axs[1].scatter(X[~mal, 0], X[~mal, 1],marker='_',cmap="red")

# Quality Metrics

Typically our goal in classification to try and accurately classify all classes. That is, we are usually interested in measuring the accuracy:

$$Accuracy = \frac{\#\text{ Correct Predictions}}{\#\text{ Data Points}}$$


But in this particular context it is important to consider the two types of errors separately:

$$\text{Type 1 Error /}  \text{ False Positive} = \text{model predicts True while the actual label is False} $$
$$\text{Type 2 Error /}  \text{ False Negative} = \text{model predicts False while the actual label is True} $$

### Question 2:
- In the context of cancer biopsies, what does each type of error mean?
- Which type of error do we care about more?

### Answer:
- A false positive means a patient who doesn't have a malignant tumor is being told that the test results were malignant. In this context, a false positive will likely result in the patient seeking more tests to confirm before starting any treatment. This is not an ideal outcome, but we are more concerned with...
- A false negative means a patient who does have breast cancer is being told that the test results were benign. This may lead to the patient ignoring the problem until it becomes worse or entirely untreatable. 

Precision and recall both relate to the number of true positive observations, but calculate two different rates. 

<b>Precision:</b> refers to the ability to ensure positives predictions are truly positive. With an algorithm that is very precise, you can be assured that most positive predictions are actually positive.

$$Precision = \frac{\#\text{ True Positives}}{\#\text{ True Positives} + \#\text{ False Positives}}$$

<b>Recall:</b> refers to the ability to ensure that cases that are positive are predicted positively. With an algorithm with high recall, you can be assured that you will predict most positive cases correctly.

$$Recall = \frac{\#\text{ True Positives}}{\#\text{ True Positives} + \#\text{ False Negatives}}$$

### Question 3:
- In the context of this data set, which are we more concerned with, and why? 
- Can we create a predictor that scores perfectly on that metric while still being useless?

### Answer 
- We care more about our model having a high recall. That way we can ensure that anyone who does have breast cancer is properly informed. 
- At the same time, we could just classify every case as malignant and achieve a recall of 1, but there is literally no point in taking that test as it offers no value.

### Example

Let's look at a simple decision tree on this data. We will use a tree of depth of 6.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtm = DecisionTreeClassifier(max_depth=6, random_state=1)
dtm.fit(X_train, y_train)

Let us now visualize the confusion matrix for how our model classifies our train data. First some boiler-plate code for visualizing the confusion matrix:

In [ ]:
from sklearn.metrics import confusion_matrix as cm

def plot_confusion_matrix(model, X, y_true, threshold=None):
    
  if threshold is None:
    y_pred = model.predict(X)
  else:
    y_pred = ((model.predict_proba(X)[:, 1] >= threshold) - .5) * 2 # transforming probabilities to -1 and 1.
    
  cm_data = cm(y_true, y_pred)
  cm_data = cm_data.ravel()[::-1].reshape(cm_data.shape)
  plt.figure(figsize=(8, 8), dpi=80)
  sns.heatmap(cm_data, annot=True, fmt=".0f", xticklabels=['Pred True', 'Pred False'],
              yticklabels = ['Actual True', 'Actual False'])
  return cm_data

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix as cm

dtm = DecisionTreeClassifier(max_depth=6, random_state=1)
dtm.fit(X_train, y_train)

cm_data = plot_confusion_matrix(dtm, X_train, y_train)

### Question 4:
- Take a moment to try and calculate the precision and recall for this training data.

### Answer:
- Run code below

In [ ]:
def print_prec_and_recall(cm_data):
  #We can quickly calculate the precision and recall from the confusion matrix
  print("Precision: ",np.round(cm_data[0, 0]/(cm_data[0, 0]+cm_data[1, 0]),5))
  print("Recall:", np.round(cm_data[0, 0]/(cm_data[0, 0]+cm_data[0, 1]),5))
print_prec_and_recall(cm_data)

Let us now visualise our the classifier's predictions on the train set:

In [ ]:
def visualize_predictions(model, X, y_true, pos_vec, title, threshold=None, stages=None, size=None):
    
    if stages is not None:
        y_pred = [((yp[:, 1] >= .5) - .5) * 2  if yp.ndim == 2 else yp
                  for i, yp in enumerate(model.staged_predict_proba(X)) if i in stages]
    elif threshold is not None:
        y_pred = ((model.predict_proba(X)[:, 1] >= threshold) - .5) * 2 # transforming probabilities to -1 and 1.
    else:
        y_pred = model.predict(X)
    
    if stages is not None:
        correct_vec = [y_true==yp for yp in y_pred]
    else:
        correct_vec = y_true==y_pred
    
    grid=np.meshgrid(np.arange(-.5,3.2,.008),np.arange(2.4,4.0,.008))
    grid=np.stack([grid[0].flatten(),grid[1].flatten()], axis=1)

    norm = colors.Normalize(vmin=-1, vmax=1)
    
    if stages is not None:
        grid_colors = [plt.cm.seismic(norm((gp[:, 1]-.5)*(-2))) if gp.ndim == 2 else plt.cm.seismic(norm(gp))
                       for i, gp in enumerate(model.staged_predict_proba(grid)) if i in stages]
    else:
        probs = model.predict_proba(grid)
        if probs.ndim == 2:
            probs = probs[:, 1]
            grid_colors = plt.cm.seismic(norm((probs-.5)*(-2)))
        else:
            grid_colors = plt.cm.seismic(norm(probs))
    

    if size is None:
        size = 200
        
    if stages is not None:
        
        fig, axs = plt.subplots(len(stages)//2, 2, figsize=(16, 7.5 * (len(stages)//2)), dpi=80)
        
        if type(size) == int:
            size_c, size_ic = len(stages) * [size//4], len(stages) * [size]
        else:
            size_c, size_ic = size.copy(), size.copy()
        
        for i, (stage, cv, gc, s_c, s_ic) in enumerate(zip(stages, correct_vec, grid_colors, size_c, size_ic)):
            
            i, j = i//2, i%2
            axs[i, j].set_title("AdaBoost stage " + str(stage))
            axs[i, j].set_xlabel("perimeter error")
            axs[i, j].set_ylabel("worst texture")
            
            axs[i, j].scatter(grid[:, 0],grid[:, 1], marker='.',c=gc, alpha=.25)
            
            if type(s_c) == int:
                
                axs[i, j].scatter(X[pos_vec & cv, 0], X[pos_vec & cv, 1], s=s_c, marker='+',c='cyan', alpha=.6)
                axs[i, j].scatter(X[~pos_vec & cv, 0], X[~pos_vec & cv, 1], s=s_c, marker='_',c="orange", alpha=.6)
                axs[i, j].scatter(X[pos_vec & ~cv, 0], X[pos_vec & ~cv, 1], s=s_ic, marker='+',c='cyan')
                axs[i, j].scatter(X[~pos_vec & ~cv, 0], X[~pos_vec & ~cv, 1], s=s_ic, marker='_',c='orange')
                
            else:
                
                axs[i, j].scatter(X[pos_vec & cv, 0], X[pos_vec & cv, 1], s=s_c[pos_vec & cv], marker='+',c='cyan', alpha=.6)
                axs[i, j].scatter(X[~pos_vec & cv, 0], X[~pos_vec & cv, 1], s=s_c[~pos_vec & cv], marker='_',c="orange", alpha=.6)
                axs[i, j].scatter(X[pos_vec & ~cv, 0], X[pos_vec & ~cv, 1], s=s_ic[pos_vec & ~cv], marker='+',c='cyan')
                axs[i, j].scatter(X[~pos_vec & ~cv, 0], X[~pos_vec & ~cv, 1], s=s_ic[~pos_vec & ~cv], marker='_',c='orange')

    else:
        plt.figure(figsize=(8, 8), dpi=80)
    
        plt.title(title)
        plt.xlabel("log perimeter error")
        plt.ylabel("log worst texture")
    
        plt.scatter(grid[:, 0],grid[:, 1], marker='.',c=grid_colors, alpha=.25)
        plt.scatter(X[pos_vec & correct_vec, 0], X[pos_vec & correct_vec, 1], marker='+',c='cyan', alpha=.6)
        plt.scatter(X[~pos_vec & correct_vec, 0], X[~pos_vec & correct_vec, 1], marker='_',c="orange", alpha=.6)
        plt.scatter(X[pos_vec & ~correct_vec, 0], X[pos_vec & ~correct_vec, 1], s=size, marker='+',c='cyan')
        plt.scatter(X[~pos_vec & ~correct_vec, 0], X[~pos_vec & ~correct_vec, 1], s=size, marker='_',c='orange')
    
visualize_predictions(dtm, X_train, y_train, mal_train, title="Train Data Classification")

### Question 5:
- Do you see any evidence that the model is overfitting?
- We can see from the color coding that different regions are colored differently. We do this to convey that the classifier has different confidences in its predictions for different regions. What does each region correspond to and how are these confidences computed?

### Answer:
- The very thin lines which classify one or two points that are deep into the regions corresponding to the other label are indications of overfitting.
- Each region corresponds to a leaf(terminal node) in the tree. At each leaf, we will have $N_{neg}$ negative training data points and $N_{pos}$ positive training data points. The model computes $p_{neg} = \frac{N_{neg}}{N_{neg} + N_{pos}}$, the probability that points in the region are negative, and $p_{pos} = 1 - p_{neg}$, the probability that the points in the region are positive. Thus, the confidence is how close the greater of the two probabilities is to 1.

In [ ]:
cm_data = plot_confusion_matrix(dtm, X_test, y_test)
print_prec_and_recall(cm_data)
visualize_predictions(dtm, X_test, y_test, mal_test, title="Test Data Classification")

We can see here that the precision and recall are considerably lower, which is an indication that we are likely overfitting.

### Question 6:
- Maintaining this model, is there a way we can improve the recall? If we can, what effect will this have on the precision?

### Answer:
- We can change the threshold that we use to classify points as positive. This will improve recall, but will lower precision.

In [ ]:
#IE: we will classify as malignant if more than 20% of points on a leaf are malignant
cm_data = plot_confusion_matrix(dtm, X_test, y_test, threshold=0.20)
print_prec_and_recall(cm_data)
visualize_predictions(dtm, X_test, y_test, mal_test, title="Test Data Classification with 0.2 threshold", threshold=0.20)

# AdaBoost

AdaBoost is founded on the <i>boosting</i> principle: the idea that we can build a strong classifier by combining a bunch of weak classifiers(also called base classifiers). The AdaBoost algorithm relies on decision stumps(trees that only consist of the root) as the base classifiers. The decision stumps are added sequentially to the main classifier and weighted according to their accuracy. The algorithm proceeds as follows.

Suppose we are given a data set $\mathcal{D} = \{(x_1, y_1), \dots, (x_n, y_n)\}$.

Initialization:
* Assign each point a *data weight* $\alpha_i=\frac{1}{n}$.

for $t$ in $[1,2,...,T]$:
* Learn a decision stump $\hat f_t(x)$. That is, find the best feature(and potentially the best split point for continuous features) to split the data on. By the "best", we mean the feature(and potentially the split point) which minimizes the $WeightedError$ below. 
* Calculate the $WeightedError$ for the learned decision stump $\hat f_t(x)$.

$$WeightedError = \frac{\sum_{i=1}^n \alpha_i I(\hat f_t(x_i) \neq y_i)}{\sum_{i=1}^n\alpha_i}$$

*   Set the *model weight* according to the weighted error
$$\hat{w}_t=\frac{1}{2}ln\left(\frac{1-WeightedError(\hat{f}_t)}{WeightedError(\hat{f}_t)}\right)$$

* Update the *data weights* $\alpha_i$, shrinking the weight corresponding to correctly classified points and raising it for the points that were incorrectly classified. Note that $-y_if_t(x_i) = -1$ if the prediction is correct and $-y_if_t(x_i) = 1$ otherwise.
$$\alpha_i=\alpha_ie^{-y_if_t(x_i)\hat{w}_t}$$

- To avoid the weights becoming too extreme and causing numerical issues with computation, we can normalized the $\alpha_i$'s at each step $t$.

To make predictions we use the final model:

$$\hat{y}=\hat{F}(x)=sign\left(\sum_{t=1}^T\hat{w}_t\hat{f}_t(x)\right)$$

Because the AdaBoost algorithm doesn't let you see things like *data weights* as the model is fitting, I wrote my own. Let's work through the code together.

In [ ]:
class AdaBoostManual:
  def __init__(self, num_trees, X, y):
    
    self.stumps=[DecisionTreeClassifier(max_depth=1,random_state=i) for i in range(num_trees)]
    self.data_weights=np.zeros((num_trees,X.shape[0]))
    self.model_weights=np.zeros(num_trees)
    
    self.X = X
    self.y = y
    self.num_trees=num_trees
    self.stump_predictions=np.zeros((num_trees,X.shape[0]))
    self.predictions=np.zeros((num_trees,X.shape[0]))
    self.psuedoprob=np.zeros((num_trees,X.shape[0]))
    
    #I just like to save lots of stuff in case I want to use it later.

  def train(self):
    
    #Initialize for the first tree - data weights are equal.
    self.data_weights[0]= 1 / self.X.shape[0]
    
    #The decision stump is fit and a number of variables are saved
    self.stumps[0].fit(self.X,self.y)
    self.stump_predictions[0]=self.stumps[0].predict(self.X)
    
    #We calculate the errors for this model(+1 if there is no error, -1 if there is an error)
    errors = self.stump_predictions[0] * self.y
    #Which we use to calculate the weighted error (unweighted for the first stump)
    WeightedError=np.sum(errors == -1)/self.X.shape[0]
    #We assign the model weight according to the weighted error
    self.model_weights[0]=0.5*np.log((1-WeightedError)/WeightedError)
    
    #We use the errors to adjust the weights
    self.data_weights[0]= self.data_weights[0] * np.exp(-errors*self.model_weights[0])
    
    #Lastly, we normalize
    self.data_weights[0]=self.data_weights[0]/self.data_weights[0].sum()
    
    # compute the predictions
    self.predictions[0] = self.model_weights[0] * self.stump_predictions[0]
    self.psuedoprob[0] = self.predictions[0]/np.max(np.abs(self.predictions[0]))
    
    for i in range(1,self.num_trees):
      
      self.stumps[i].fit(self.X,self.y,sample_weight=self.data_weights[i-1])
      self.stump_predictions[i] = self.stumps[i].predict(self.X)
    
      #We take the sign of the resulting sum as the predicted value
      errors=self.stump_predictions[i] * self.y
      #Which we use to calculate the weighted error (unweighted for the first stump)
      WeightedError= np.sum(self.data_weights[i-1] * (errors == -1))/self.data_weights[i-1].sum()
      #Then we again assign the model weight and update data weights
      self.model_weights[i]=0.5*np.log((1-WeightedError)/WeightedError)
    
      #We use the errors to adjust the weights
      self.data_weights[i]=self.data_weights[i-1]*np.exp(-errors*self.model_weights[i])
    
      #Lastly, we normalize
      self.data_weights[i]=self.data_weights[i]/self.data_weights[i].sum()
        
      # compute the predictions
      self.predictions[i] = self.predictions[i - 1] + self.model_weights[i] * self.stump_predictions[i]
      self.psuedoprob[i] = self.predictions[i]/np.max(np.abs(self.predictions[i]))

  def predict(self,X,early_stop=None):
    
    preds=np.zeros(X.shape[0])
    if early_stop is None:
      for i in range(self.num_trees):
        tmp=self.stumps[i].predict(X)
        preds=preds+tmp*self.model_weights[i]
    else:
      for i in range(early_stop):
        tmp=self.stumps[i].predict(X)
        preds=preds+tmp*self.model_weights[i]
    preds=np.sign(preds)
    return(preds)

  def staged_predict_proba(self, X):
        preds=np.zeros((self.num_trees, X.shape[0]))
        for i in range(self.num_trees):
            tmp=self.stumps[i].predict(X)
            if i == 0:
                preds[i]= tmp * self.model_weights[i]
            else:
                preds[i]= preds[i-1] + tmp*self.model_weights[i]
        preds=preds/np.max(np.abs(preds))
        return(-preds)
        
  #This is not a true probability, but instead returns a value between -1 and 1.
  def predict_proba(self,X,early_stop=None):
    preds=np.zeros(X.shape[0])
    if early_stop is None:
      for i in range(self.num_trees):
        tmp=self.stumps[i].predict(X)
        preds=preds+tmp*self.model_weights[i]
    else:
      for i in range(early_stop):
        tmp=self.stumps[i].predict(X)
        preds=preds+tmp*self.model_weights[i]
    preds=preds/np.max(np.abs(preds))
    return(-preds)

ABE=AdaBoostManual(48, X_train, y_train)
ABE.train()


Let's see how well our model is doing:

In [ ]:
cm_data = plot_confusion_matrix(ABE, X_test, y_test)
print_prec_and_recall(cm_data)
visualize_predictions(ABE, X_test, y_test, mal_test, title="Test Data Classification")

Let's look at how weights are changing from run to run

The plots below show how the points are being classified after the first $k$ trees. The size of a point is proportional to the weight of the point after the $k$th tree, with the incorrectly classified points emphasized. 

In [ ]:
size = ABE.data_weights
size = (size - np.min(size))/(np.max(size) - np.min(size))
visualize_predictions(ABE, X_train, y_train, mal_train, title="Train Data Classification", stages=range(0, 48, 6), size=size * 600)

The *sklearn* library of course has its own AdaBoost algorithm, capable of either classification or regression. Much easier than writing your own, though it does not give quick and easy access to some of the parameters (such as the model or data weights). 

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abc=AdaBoostClassifier(n_estimators=48, random_state=1)
abc.fit(X_train, y_train)

cm_data = plot_confusion_matrix(abc, X_train, y_train)
print_prec_and_recall(cm_data)
visualize_predictions(abc, X_train, y_train, mal_train, title="Train Data Classification")

In [ ]:
#testing data
cm_data = plot_confusion_matrix(abc, X_test, y_test)
print_prec_and_recall(cm_data)
visualize_predictions(abc, X_test, y_test, mal_test, title="Test Data Classification")

We can even look at the stagewise predictions for only the first $k$ models, but the access is somewhat clunky. Below is the 10th model.

In [ ]:
visualize_predictions(abc, X_train, y_train, mal_train, title="Train Data Classification", stages=range(0, 48, 6))

# Random Forests

Random Forests attempt to improve upon using a single Decision Tree. The RF algorithm does so by training an ensemble of trees via Bootstrapping. The RF then uses each trained tree to cast a voter and the majority decision will represent the classifier's decision.

To produce a variety of trees from a single data set, we use BootStrapping which consists of sampling multiple new training data sets from the original data set and training a tree for each of these. Furthermore, by randomly selecting a subset of all the features on which to construct the trees, we further increase the variety of trees which we train. This helps considerably with reducing the variance inherent to training a single Decision Tree. 

##### Random Forrest

<img src="https://occ-0-2794-2219.1.nflxso.net/dnm/api/v6/E8vDc_W8CLv7-yMQu8KMEC7Rrr8/AAAABRH4YM-SODj1fGal_ylc15ebTe90p_j_WPiMUjXpRGFZqMdCtc7zSFJe9C2OliUhCGmyu0APQWKZ3Q6CzAGSA8_D26Sj.jpg" width="400"/>

Let us now see how we can train a Random Forest in sklearn

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier(n_estimators=50, random_state=1, max_depth=6)
rfc.fit(X_train, y_train)

cm_data = plot_confusion_matrix(rfc, X_train, y_train)
print_prec_and_recall(cm_data)
visualize_predictions(rfc, X_train, y_train, mal_train, title="Train Data Classification")

In [ ]:
#Test data
cm_data = plot_confusion_matrix(rfc, X_test, y_test)
print_prec_and_recall(cm_data)
visualize_predictions(rfc, X_test, y_test, mal_test, title="Test Data Classification")